In [1]:
import sys
sys.path.append("../script/")
import csv
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from math import floor
import pandas as pd
import ClassiferHelperAPI as CH
import importlib
importlib.reload(CH)
from sklearn import metrics
from collections import OrderedDict
import math
from sklearn.metrics import classification_report,accuracy_score,f1_score,precision_score,recall_score,roc_auc_score,mean_absolute_error, mean_squared_error,zero_one_loss

In [2]:
pd.DataFrame.from_csv("../FinalResults/rankListImages_expt2.csv")

with open("../FinalResults/rankListImages_expt2.csv","r") as inpFl:
    reader = csv.reader(inpFl)
    head = reader.__next__()
    data = [row for row in reader]

In [3]:
propCount = [float(row[4]) for row in data]
distribFnc = dict(Counter(propCount))
# df = pd.DataFrame(cdf,index=['Counter']).transpose()

In [4]:
# plotting histograms - RELEVANT, do not delete, but incorrect
bins = [0,10,20,30,40,50,60,70,80,90,100]

a,b,c = plt.hist(propCount,bins,histtype='bar')
plt.ylabel("Number of images")
plt.xlabel("Share proportion")
plt.title("Share proportion(X-axis) versus Number of Images shared x% of times(Y-axis)")
plt.yticks(np.arange(0, 200+1, 10.0))
plt.grid(b=True, which='major', color='k', linestyle=':')
plt.savefig("../ClassifierResults/shareProportionVsNumImgsMod.png",bbox_inches='tight')

In [47]:
bucket = dict.fromkeys([i for i in range(0,110,10)],[])

for prop in propCount:
    key = math.ceil(prop/10) * 10
    bucket[key] = bucket.get(key,[]) + [prop] 

bucketCount = {}
for key in bucket:
    bucketCount[key] = len(bucket[key])
    
df = pd.DataFrame(bucketCount,index=['y']).transpose()
df.plot()
plt.ylabel("Number of images")
plt.xlabel("Share proportion")
plt.title("Share proportion(X-axis) versus Number of Images shared x% of times(Y-axis)")
plt.xticks(np.arange(0,100+1,10))
plt.yticks(np.arange(0, 200+1, 10.0))
plt.grid(b=True, which='major', color='k', linestyle=':')
plt.savefig("../ClassifierResults/shareProportionVsNumImgsMod.png",bbox_inches='tight')


In [37]:
methods = ['logistic','svm','dtree','random_forests']
fl = open("/tmp/extrmClfReport.dat","w")
perfMetricDict = {}
performance_metric_dict = []
for method in methods:
    for i in np.arange(0.1,1,0.1):
        temp = OrderedDict()
        clf,test_x, test_y, predics, pred_prob = CH.buildBinClassifier(data,allAttribs,i,80,method,True)
        accScore = accuracy_score(test_y,predics)
        f1Score = f1_score(test_y,predics)
        precision = precision_score(test_y,predics)
        recall = recall_score(test_y,predics)
        auc = roc_auc_score(test_y,pred_prob)
        abserr = mean_absolute_error(test_y,predics)
        sqerr = mean_squared_error(test_y,predics)
        zerooneloss = zero_one_loss(test_y,predics)
        temp['Classifier'] = method
        temp['Train-Test Split'] = i * 100
        temp['Accuracy'] = accScore
        temp['Precision'] = precision
        temp['Recall'] = recall
        temp['AUC'] = auc
        temp['F1 score'] = f1Score
        temp['Squared Error'] = sqerr
        performance_metric_dict.append(temp)
        clfDict[(method,i)] = clf
        fl.write(str("Method: " + method + "\nSplit Ratio: " + str(i) +"\n\n"))
        fl.write(classification_report(test_y,predics))
        
fl.close()

/Users/sreejithmenon/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/sreejithmenon/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/sreejithmenon/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [38]:
df = pd.DataFrame(performance_metric_dict)

df = df[['Classifier','Train-Test Split','Accuracy','Precision','Recall','F1 score','AUC','Squared Error']]
df.to_csv("../ClassifierResults/perform_metrics_extrmClf1.csv",index=False)
df

,Classifier,Train-Test Split,Accuracy,Precision,Recall,F1 score,AUC,Squared Error
0,logistic,10,0.767442,0.800000,0.800000,0.800000,0.788889,0.232558
1,logistic,20,0.705882,0.734694,0.750000,0.742268,0.754505,0.294118
2,logistic,30,0.740157,0.740260,0.814286,0.775510,0.779449,0.259843
3,logistic,40,0.715976,0.711538,0.804348,0.755102,0.774139,0.284024
4,logistic,50,0.710900,0.697842,0.836207,0.760784,0.754991,0.289100
5,logistic,60,0.712598,0.704819,0.829787,0.762215,0.760215,0.287402
6,logistic,70,0.702703,0.740506,0.713415,0.726708,0.744041,0.297297
7,logistic,80,0.656805,0.673367,0.724324,0.697917,0.707613,0.343195
8,logistic,90,0.631579,0.719512,0.556604,0.627660,0.704220,0.368421
9,svm,10,0.581395,0.581395,1.000000,0.735294,0.797778,0.418605


In [6]:
clf,test_x, test_y, predics, pred_prob,score = CH.buildBinClassifier(data,allAttribs,0.4,80,'logistic',True)

In [88]:
for key in clfDict:
    clf = clfDict[key]
    
    if key[0] == 'random_forests':
        print(key, clf.estimators_)
        break
        
print(len(test_x.columns))

('random_forests', 0.20000000000000001) [DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1865640145, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=274807701, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1148070046, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='auto', ma

In [65]:
d = OrderedDict.fromkeys(test_x.columns,0)

for i in range(len(clf.coef_[0])):
    d[test_x.columns[i]] = clf.coef_[0][i]


In [69]:
pd.DataFrame(d,index=['co-efficients']).transpose()

,co-efficients
zebra_plains,-0.468173
giraffe_masai,0.516168
UNKNOWN NAME,-0.043396
Female,0.498140
UNKNOWN SEX,-0.409351
Male,-0.210480
unknown,-0.406869
juveniles - one year old,-0.351691
juveniles- two year old,-0.268446
adult,0.034764


In [34]:
d = CH.createDataFlDict(data,allAttribs,True,80,True)

In [35]:
# number of positive/negative examples:

posEgs = 0
negEgs = 0

for key in d.keys():
    if d[key]['TARGET'] == 1:
        posEgs += 1
    else:
        negEgs += 1

In [36]:
posEgs,negEgs

(230, 192)

In [19]:
print(len(list(filter(lambda x : float(data[x]['Proportion']) >= 80.0,data.keys()))))
print(len(list(filter(lambda x : float(data[x]['Proportion']) <= 20.0,data.keys()))))

230
192


In [20]:
230 + 192

422

In [21]:
fDataKeys= list(filter(lambda x : float(data[x]['Proportion']) >= 80.0 or float(data[x]['Proportion']) <= 20.0,data.keys()))
fData = {gid: data[gid] for gid in fDataKeys}

In [30]:
pd.DataFrame(fData).transpose()['Proportion'].to_csv("/tmp/props.csv")

In [32]:
pd.DataFrame(d).transpose()['TARGET'].to_csv("/tmp/targs.csv")

In [50]:
methods = ['logistic','svm','dtree','random_forests']
fl = open("/tmp/extrmClfReport.dat","w")
perfMetricDict = {}
performance_metric_dict = []
for method in methods:
    for i in np.arange(0.1,1,0.1):
        temp = OrderedDict()
        clf,test_x, test_y, predics, pred_prob = CH.buildBinClassifier(data,allAttribs,i,80,method,True)
        temp = CH.evalClassifierPerf(clf,test_x,test_y,predics,pred_prob)
        temp['Method'] = method
        temp['Train-Test Split'] = i * 100
        performance_metric_dict.append(temp)
        clfDict[(method,i)] = clf
        fl.write(str("Method: " + method + "\nSplit Ratio: " + str(i) +"\n\n"))
        fl.write(classification_report(test_y,predics))
        
fl.close()

/Users/sreejithmenon/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/sreejithmenon/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/sreejithmenon/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


KeyboardInterrupt: 

In [2]:
import mpl_toolkits as mt

In [5]:
from mpl_toolkits.basemap import Basemap

ImportError: No module named 'mpl_toolkits.basemap'